In [1]:
# Official packages
import os
import copy
import math

# Third-party packages
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm

# In-house packages
from BrukerSpSpEpi import BrukerSpSpEpiExp


IndentationError: expected an indented block (BrukerSpSpEpi.py, line 207)

In [ ]:


def zerofill_fid_2d(fid_2d):
    nbr_zerofill_lines = spsp_epi._exp_data_dim_dict['dim_r_image_ph'] - spsp_epi._exp_data_dim_dict['dim_k_raw_ph']
    zerofilled_fid_2d = np.pad(fid_2d, ((nbr_zerofill_lines, 0),(0, 0)), 'constant', constant_values=(0))
    return zerofilled_fid_2d

def split_fid_2d(fid_2d):
    fid_left = fid_2d[...,:spsp_epi._exp_data_dim_dict['dim_k_raw_ro']:]
    fid_right = fid_2d[...,spsp_epi._exp_data_dim_dict['dim_k_raw_ro']::]
    return fid_left, fid_right

def mirror_odd_lines(fid_2d):
    mirrored_fid_2d = copy.deepcopy(fid_2d)
    for line_nbr, line in enumerate(mirrored_fid_2d):
        if (line_nbr % 2 ):
             mirrored_fid_2d[line_nbr] = line[::-1,...]
    return mirrored_fid_2d

def synthesize_phase_conjugate_fid_2d(fid_2d):
    synthesized_fid_2d = copy.deepcopy(fid_2d)
    for line_idx, line in enumerate(fid_2d):
        if (not line.any()):
            line = zerofilled_fid_2d[-1*line_idx]
            synthesized_fid_2d[line_idx] = line[::-1]
    return synthesized_fid_2d

def align_echo_center(fid_2d):
    aligned_fid_2d = copy.deepcopy(fid_2d)
    for idx, line in enumerate(fid_2d):
        shift = 60 - np.argmax(np.abs(line))
        aligned_fid_2d[idx] = np.roll(line, shift)
    return aligned_fid_2d

def _normalize(array):
    return array/np.max(array)

def view_epi_slice_3d(spsp_epi_exp):
    
    fig = plt.figure()
    ax = fig.gca(projection='3d')
    #X, Y, Z = axes3d.get_test_data(0.01)

    _x = np.arange(0, spsp_epi_exp._exp_data_dim_dict['dim_r_image_ro'], 1 )
    _y = np.arange(0, spsp_epi_exp._exp_data_dim_dict['dim_r_image_ph'], 1)

    X, Y = np.meshgrid(_x, _y)
    Z = _normalize(spsp_epi_exp.r_space_data['Mag'])

    # Plot the 3D surface
    ax.plot_surface(X, Y, Z, cmap=cm.RdGy_r, linewidth=0.5, antialiased=False,alpha=0.25)

    # Plot projections of the contours for each dimension.  By choosing offsets
    # that match the appropriate axes limits, the projected contours will sit on
    # the 'walls' of the graph
    cset = ax.contour(X, Y, Z, zdir='z', offset=-.1, cmap=cm.seismic, alpha=0.99)

    x = np.arange(0, spsp_epi_exp._exp_data_dim_dict['dim_r_image_ro'], 1 )
    y = _normalize(spsp_epi_exp.magnitude_proj['ph'])
    ax.plot(x, y, zs=0, zdir='x', label='curve in (x, y)')

    x = np.arange(0, spsp_epi_exp._exp_data_dim_dict['dim_r_image_ph'], 1 )
    y = _normalize(spsp_epi_exp.magnitude_proj['ro'])
    ax.plot(x, y, zs=120, zdir='y', label='curve in (x, y)')

    ax.set_xlim(-0, 120)
    ax.set_ylim(-0, 120)
    ax.set_zlim(-.1, 1)

    ax.set_xlabel('Ph-Direction')
    ax.set_ylabel('RO-Direction')
    ax.set_zlabel('Normalized Intensity [1]')

    plt.show()



def process_fid(raw_fid):
    test_fid = np.array(np.array_split(raw_fid,16))
    k_space = {}
 
    k_space['Pos'] = np.squeeze(test_fid[...,:16:])
    k_space['Neg'] = np.squeeze(test_fid[...,16::])
    
    r_space["Pos"] = np.fft.fftshift(np.fft.fft2(k_space["Pos"]))
    r_space["Neg"] = np.fft.fftshift(np.fft.fft2(k_space["Neg"]))
    r_space["Abs"]  = np.abs(r_space['Pos']) + np.abs(r_space['Neg'])

    return k_space, r_space

In [ ]:
calib_exp_dir = "../20230305_163320_AgroseCylinder2_1_1/21"
        
spsp_epi_calib = BrukerSpSpEpiExp(calib_exp_dir, metabolite_list = ['Urea'])

overflow_exp_dir = "../LTL610_014_HP_spspEPI_6.0.1//20"

spsp_epi_overflow = BrukerSpSpEpiExp(overflow_exp_dir)

In [ ]:
print(spsp_epi_calib.transient_space['Urea'].dtypes)